In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

2022-12-07 10:12:22.772857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 10:12:22.860537: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 10:12:22.860551: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-07 10:12:22.875003: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 10:12:23.306708: W tensorflow/stream_executor/platform/de

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-07 10:12:24.500202: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-07 10:12:24.500316: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 10:12:24.500365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-07 10:12:24.500400: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-07 10:12:24.500434: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 51)                867       
                                                                 
Total params: 17,411
Trainable params: 17,411
Non-trainable params: 0
____________________________________________________

In [8]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)



Epoch 1/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9318 - accuracy: 0.0138
Epoch 2/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9290 - accuracy: 0.0276
Epoch 3/550
5/5 [==============================] - 0s 1ms/step - loss: 3.9272 - accuracy: 0.0276
Epoch 4/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9249 - accuracy: 0.0276
Epoch 5/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9227 - accuracy: 0.0276
Epoch 6/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9199 - accuracy: 0.0276
Epoch 7/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9175 - accuracy: 0.0276
Epoch 8/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9146 - accuracy: 0.0276
Epoch 9/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9112 - accuracy: 0.0276
Epoch 10/550
5/5 [==============================] - 0s 2ms/step - loss: 3.9077 - accuracy: 0.0345
Epoch 11/550
5/5 [===========

5/5 [==============================] - 0s 2ms/step - loss: 2.6258 - accuracy: 0.2621
Epoch 85/550
5/5 [==============================] - 0s 2ms/step - loss: 2.6043 - accuracy: 0.2345
Epoch 86/550
5/5 [==============================] - 0s 2ms/step - loss: 2.5833 - accuracy: 0.2414
Epoch 87/550
5/5 [==============================] - 0s 2ms/step - loss: 2.5603 - accuracy: 0.2414
Epoch 88/550
5/5 [==============================] - 0s 2ms/step - loss: 2.5388 - accuracy: 0.2483
Epoch 89/550
5/5 [==============================] - 0s 2ms/step - loss: 2.5163 - accuracy: 0.2690
Epoch 90/550
5/5 [==============================] - 0s 2ms/step - loss: 2.4950 - accuracy: 0.2828
Epoch 91/550
5/5 [==============================] - 0s 1ms/step - loss: 2.4724 - accuracy: 0.2966
Epoch 92/550
5/5 [==============================] - 0s 2ms/step - loss: 2.4515 - accuracy: 0.3034
Epoch 93/550
5/5 [==============================] - 0s 1ms/step - loss: 2.4298 - accuracy: 0.3172
Epoch 94/550
5/5 [===============

5/5 [==============================] - 0s 2ms/step - loss: 0.9207 - accuracy: 0.8621
Epoch 168/550
5/5 [==============================] - 0s 2ms/step - loss: 0.9084 - accuracy: 0.8483
Epoch 169/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8917 - accuracy: 0.8552
Epoch 170/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8789 - accuracy: 0.8552
Epoch 171/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8658 - accuracy: 0.8690
Epoch 172/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8538 - accuracy: 0.8759
Epoch 173/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8430 - accuracy: 0.8897
Epoch 174/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8302 - accuracy: 0.8828
Epoch 175/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8171 - accuracy: 0.8828
Epoch 176/550
5/5 [==============================] - 0s 2ms/step - loss: 0.8030 - accuracy: 0.8759
Epoch 177/550
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.2739 - accuracy: 0.9931
Epoch 250/550
5/5 [==============================] - 0s 1ms/step - loss: 0.2703 - accuracy: 0.9931
Epoch 251/550
5/5 [==============================] - 0s 1ms/step - loss: 0.2672 - accuracy: 0.9931
Epoch 252/550
5/5 [==============================] - 0s 2ms/step - loss: 0.2624 - accuracy: 0.9931
Epoch 253/550
5/5 [==============================] - 0s 2ms/step - loss: 0.2583 - accuracy: 0.9931
Epoch 254/550
5/5 [==============================] - 0s 2ms/step - loss: 0.2546 - accuracy: 0.9931
Epoch 255/550
5/5 [==============================] - 0s 2ms/step - loss: 0.2529 - accuracy: 0.9931
Epoch 256/550
5/5 [==============================] - 0s 1ms/step - loss: 0.2484 - accuracy: 0.9931
Epoch 257/550
5/5 [==============================] - 0s 2ms/step - loss: 0.2454 - accuracy: 0.9931
Epoch 258/550
5/5 [==============================] - 0s 1ms/step - loss: 0.2415 - accuracy: 0.9931
Epoch 259/550
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.0968 - accuracy: 1.0000
Epoch 332/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0955 - accuracy: 1.0000
Epoch 333/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0947 - accuracy: 1.0000
Epoch 334/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0932 - accuracy: 1.0000
Epoch 335/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0924 - accuracy: 1.0000
Epoch 336/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0916 - accuracy: 1.0000
Epoch 337/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0913 - accuracy: 1.0000
Epoch 338/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0894 - accuracy: 1.0000
Epoch 339/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0887 - accuracy: 1.0000
Epoch 340/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0875 - accuracy: 1.0000
Epoch 341/550
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.0417 - accuracy: 1.0000
Epoch 414/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0414 - accuracy: 1.0000
Epoch 415/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0410 - accuracy: 1.0000
Epoch 416/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0403 - accuracy: 1.0000
Epoch 417/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0401 - accuracy: 1.0000
Epoch 418/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0397 - accuracy: 1.0000
Epoch 419/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0393 - accuracy: 1.0000
Epoch 420/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0389 - accuracy: 1.0000
Epoch 421/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0386 - accuracy: 1.0000
Epoch 422/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0383 - accuracy: 1.0000
Epoch 423/550
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 496/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0205 - accuracy: 1.0000
Epoch 497/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0204 - accuracy: 1.0000
Epoch 498/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0203 - accuracy: 1.0000
Epoch 499/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 500/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 501/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0197 - accuracy: 1.0000
Epoch 502/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0196 - accuracy: 1.0000
Epoch 503/550
5/5 [==============================] - 0s 1ms/step - loss: 0.0196 - accuracy: 1.0000
Epoch 504/550
5/5 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 505/550
5/5 [=====

In [9]:
import pickle
# saving model
model.save("chat_model")




# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets
